# DATA

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir("G:/SEM 3 DATA SCIENCE/NLP")

In [2]:
df = pd.read_csv("Consumer_Complaints (1).csv")
df.head()

C:\Users\Devyani\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/17/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,ALLY FINANCIAL INC.,TX,75035,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309495
1,07/17/2019,Credit card or prepaid card,General-purpose prepaid card,Trouble using the card,Problem using the card to withdraw money from ...,NaN,NaN,SQUARE INC,KS,674XX,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3310031
2,07/17/2019,Debt collection,Other debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Diversified Consultants, Inc.",FL,NaN,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309687
3,07/17/2019,Mortgage,VA mortgage,Trouble during payment process,NaN,NaN,NaN,"FLAGSTAR BANK, FSB",VA,22554,Servicemember,NaN,Web,07/17/2019,In progress,Yes,NaN,3308925
4,07/17/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes it acted appropriately as aut...,BYL Collection Services,TN,370XX,Servicemember,NaN,Web,07/17/2019,Closed with explanation,Yes,NaN,3308914


In [35]:
df1 = df[["Consumer complaint narrative"]]

# Preprocessing

Removing Null Values

In [36]:
df1.isnull().sum()

Consumer complaint narrative    938347
dtype: int64

In [37]:
df1 = df1.dropna()
df1.isnull().sum()

Consumer complaint narrative    0
dtype: int64

Cleaning the classes

Selecting First 100 Rows

In [38]:
df1 = df1.iloc[:100] 
df1.shape

(100, 1)

In [39]:
corpus = df1["Consumer complaint narrative"].tolist()
corpus

["!!!!!! This is not a Duplicate!!!! I have contacted XXXX XXXX and Experian about a fraudulent collection account being reported on my credit file. XXXX XXXX about a fraudulent debt that was placed on my credit report. Someone attempted to rent an apartment in my name and the fees associated with them submitting an application in my name have been attached to my credit file. I have contacted XXXX XXXX and told them directly the account they were reporting was fraud and they have been verbally abusive to me, and refused to remove the account from my credit file even though they know the collection is as a result of identity fraud. I have contacted XXXX XXXX in Experian 's Consumer Relations Department 7 times in total and left voice mail messages for him and he has never returned any of my very detailed messages. I was told by Experian to contact XXXX XXXX when I noticed information on my credit file that was fraudulent. I have done as I was asked to do, but in all of the 7 detailed me

Tokenizing, Lemmatizing and Removing Stopwords and Punctuations

In [40]:
# from nltk.corpus import stopwords
# from string import punctuation
# stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)

In [41]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Devyani\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [42]:
final_corpus= []
for i in range(len(corpus)):
    text = word_tokenize(corpus[i].lower())
    sent = " ".join(text)
    sent = " ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
    
    final_corpus.append(sent)

In [43]:
new_df = pd.DataFrame(final_corpus)

In [44]:
new_df.columns = ["Complaint"]

In [45]:
new_df.head()

,Complaint
0,! ! ! ! ! ! this is not a duplicate ! ! ! ! i ...
1,{ $ 16000 . 00 } . this is not my debt .
2,when they call they dont answer the call and t...
3,i provided account information which debt was ...
4,there are 2 on my credit report from a collect...


TF - IDF Value

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [48]:
vector= tfidf.fit_transform(new_df["Complaint"])

In [49]:
vector

<100x1190 sparse matrix of type '<class 'numpy.float64'>'
	with 5702 stored elements in Compressed Sparse Row format>

In [32]:
vector.toarray()

array([[0.        , 0.        , 0.04762862, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [51]:
X= vector.toarray()

Data Train - Test Split

In [50]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=3, n_iter=100)

In [52]:
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=3, n_iter=100,
       random_state=None, tol=0.0)

In [56]:
row1 = lsa.components_[0]

In [57]:
row1

array([0.07749813, 0.00721741, 0.00685702, ..., 0.06940822, 0.02727772,
       0.0031085 ])

In [58]:
lsa.components_.shape

(3, 1190)